In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query,run_read_prod_query

from datetime import datetime

from procurement_lib import GoogleSheet

In [2]:
todays_date = datetime.today().date()

In [3]:
paramss_csv=GoogleSheet("1cnuspMEt1MlnG7Io7Il8IiopXfJy0vmnalInhbJOQsk")
df = paramss_csv.get_as_dataframe('info_py')

# Convert to datetime
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [4]:
query = """
select
    competitor.competitor_name,
    site.identifier_value as site_code,
    quotation_date.full_date AS quotation_date,
    su.source_id,
    ROUND(AVG(cpp.product_selected_price),2)::float as price
from dpr_product_pricing.fact_collected_product_prices cpp
    inner join dpr_shared.dim_date quotation_date
        on cpp.dim_quotation_date = quotation_date.date_id
    inner join dpr_shared.dim_time quotation_time
        on cpp.dim_quotation_time = quotation_time.time_id
    inner join dpr_shared.dim_site site
        on cpp.dim_site = site.site_id
    inner join dpr_shared.dim_category cat
        on cpp.dim_category = cat.category_id
    inner join dpr_product_pricing.dim_product_outlier_type outlier_type
        on cpp.dim_outlier_type = outlier_type.outlier_type_id
    inner join dpr_product_pricing.dim_product_source_type source_type
        on cpp.dim_source_type = source_type.source_type_id
    inner join dpr_product_pricing.dim_product_competitor competitor
        on cpp.dim_competitor = competitor.competitor_id
    inner join dpr_product_pricing.dim_product_competitor_type competitor_type
        on(
            case
                when cpp.super_category = 'Fruver'
                    then competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
                when cpp.super_category = 'Multicategoría'
                    then competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
            end
        )
    inner join dpr_shared.dim_stock_unit su
        on cpp.dim_stock_unit = su.stock_unit_id
where quotation_date.full_date >= current_date - 10
    AND competitor.competitor_name NOT ILIKE '%cayena%'
    AND site.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND su.source_id IN {skus}
GROUP BY 1,2,3,4
""".format(skus=tuple(df.source_id.unique()))
df_zkkkkk = run_read_dwd_query(query)

df_zkkkkk = df_zkkkkk.dropna().reset_index(drop=True)
df_zkkkkk['lifetime'] = 8

In [5]:
# Ensure dataframe is sorted by 'quotation_date'
df_zkkkkk = df_zkkkkk.sort_values(by='quotation_date')

# Generate the required rows for missing dates
new_rows = []

for (competitor, source_id), group in df_zkkkkk.groupby(['competitor_name', 'source_id']):
    group = group.sort_values(by='quotation_date')
    last_known_price = None
    last_known_date = None
    lifetime = 8
    
    for current_index in range(len(group)):
        current_date = group.iloc[current_index]['quotation_date']
        price = group.iloc[current_index]['price']
        
        # If this is not the first iteration, fill in missing dates
        if last_known_date is not None:
            days_diff = (current_date - last_known_date).days
            if days_diff > 1:
                for j in range(1, min(days_diff, lifetime + 1)):
                    new_date = last_known_date + timedelta(days=j)
                    new_row = {
                        'site_code': group.iloc[current_index]['site_code'],
                        'quotation_date': new_date,
                        'competitor_name': competitor,
                        'source_id': source_id,
                        'price': last_known_price,
                        'lifetime': lifetime - j
                    }
                    new_rows.append(new_row)
                    
                    # Stop if we reach a new datapoint date
                    if new_date + timedelta(days=1) == current_date:
                        break
        
        # Update the last known values and reset lifetime
        last_known_price = price
        last_known_date = current_date
        lifetime = 8  # Reset lifetime

    # After processing all known dates for the group, continue generating rows until lifetime reaches 0
    while lifetime > 0:
        last_known_date += timedelta(days=1)
        new_row = {
            'site_code': group.iloc[-1]['site_code'],
            'quotation_date': last_known_date,
            'competitor_name': competitor,
            'source_id': source_id,
            'price': last_known_price,
            'lifetime': lifetime - 1
        }
        new_rows.append(new_row)
        lifetime -= 1

# Append new rows to the dataframe
df_zkkkkk = df_zkkkkk.append(new_rows, ignore_index=True)

# Sort the final dataframe
df_zkkkkk = df_zkkkkk.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
df_zkkkkk['replica'] = df_zkkkkk['lifetime'] == 8

df_bench = df_zkkkkk.copy()
df_bench['quotation_date'] = pd.to_datetime(df_bench['quotation_date'])

In [6]:
query = """
WITH RECURSIVE calendar(calendar_date) AS (
  SELECT DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day')
  UNION ALL
  SELECT calendar_date + INTERVAL '1 day'
  FROM calendar
  WHERE calendar_date BETWEEN DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day') AND DATE(GETDATE() - 1) 
),

info AS (
SELECT
    DATE(coalesce(prices.last_modified_at, prices.created_at)) as created_at,
    pp.frida_id as source_id,
    MIN(coalesce(tiers.tax_price, prices.tax_price)) as price,
    MIN(coalesce(tiers.sale_price, prices.sale_price)) as net_price

FROM postgres_growth."growth_pricing.prices_history" prices
LEFT JOIN postgres_growth."growth_pricing.price_tiers_history" tiers ON prices.id = tiers.price_history_id
LEFT JOIN postgres_growth."growth_pricing.skus" skus ON prices.sku_id = skus.id
LEFT JOIN postgres_main_co."purchase_orders.products" p ON skus.sku_id = p.frida_id
LEFT JOIN postgres_main_co."purchase_orders.products" pp ON COALESCE(p.parent_id, p.id) = pp.id

WHERE DATE(prices.created_at) >= DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day')
 AND p.region_code IN ('SPO','BHZ','CWB','VCP')
 AND p.deleted_at IS NULL
 AND prices.created_by NOT ILIKE '%CATALOG%'
 AND pp.product_category_id IN (5,6,7,8,9,10,13,18) -- 1 ES FRUVER
 AND pp.frida_id IN {skus}
GROUP BY 1,2--,3,4
),

done AS (
SELECT
  DATE(c.calendar_date) AS quotation_date,
  --s.region,
  --s.parent_product_name,
  (s.source_id)::int as source_id,
  LAG(i.price IGNORE NULLS) OVER (PARTITION BY s.source_id ORDER BY c.calendar_date)::FLOAT AS p_price_tool,
  LAG(i.net_price IGNORE NULLS) OVER (PARTITION BY s.source_id ORDER BY c.calendar_date)::FLOAT AS net_price_tool


FROM calendar c
CROSS JOIN (SELECT DISTINCT source_id FROM info) s
LEFT JOIN info i ON c.calendar_date = i.created_at-1 AND s.source_id = i.source_id
)

SELECT *
FROM done
WHERE net_price_tool IS NOT NULL
 AND quotation_date >= DATE(CURRENT_DATE - 14)
""".format(skus=tuple(df_bench.source_id.unique()))
dfp = run_read_prod_query(query)  
dfp['quotation_date'] = pd.to_datetime(dfp['quotation_date'])

In [7]:
df2 = pd.merge(df_bench, dfp.loc[dfp.quotation_date.isin([df.quotation_date.max(),todays_date])], left_on=['source_id','quotation_date'], right_on=['source_id','quotation_date'], how='inner')

In [8]:
df2 = df2.groupby(['site_code','quotation_date','source_id','p_price_tool','net_price_tool'])['price'].quantile(0.25).reset_index()

In [9]:
# Function to get min and max date rows
def get_min_max_rows(group):
    min_row = group.loc[group['quotation_date'].idxmin()]
    max_row = group.loc[group['quotation_date'].idxmax()]
    return pd.DataFrame([min_row, max_row])

# Apply the function to each group and reset the index
comparison_df = df2.groupby(['site_code', 'source_id']).apply(get_min_max_rows).reset_index(drop=True)

# Split the comparison_df into min and max DataFrames
min_df = comparison_df[comparison_df['quotation_date'] == comparison_df.groupby(['site_code', 'source_id'])['quotation_date'].transform('min')]
max_df = comparison_df[comparison_df['quotation_date'] == comparison_df.groupby(['site_code', 'source_id'])['quotation_date'].transform('max')]

# Merge min and max DataFrames for comparison
comparison_result = pd.merge(min_df, max_df, on=['site_code', 'source_id'], suffixes=('_min', '_max'))

In [10]:
comparison_result.loc[comparison_result.price_min > comparison_result.price_max]

,site_code,quotation_date_min,source_id,p_price_tool_min,net_price_tool_min,price_min,quotation_date_max,p_price_tool_max,net_price_tool_max,price_max
5,BHZ,2024-06-25,263787,9.25,9.25,9.6075,2024-06-28,9.77,9.77,9.4800
16,BHZ,2024-06-25,308307,3.26,3.26,3.2575,2024-06-28,3.28,3.28,3.1800
24,BHZ,2024-06-25,453793,8.24,7.58,7.5825,2024-06-28,8.24,7.58,7.4075
29,BHZ,2024-06-25,569807,18.01,18.01,18.0825,2024-06-28,18.01,18.01,17.9000
40,BHZ,2024-06-25,623599,2.85,2.85,2.8525,2024-06-28,2.85,2.85,2.8150
41,BHZ,2024-06-25,625775,15.99,15.99,15.9900,2024-06-28,15.99,15.99,15.8625
47,CWB,2024-06-25,369413,10.19,10.19,10.5000,2024-06-28,10.19,10.19,9.9900
52,CWB,2024-06-25,423479,7.15,7.15,7.8700,2024-06-28,7.39,7.39,7.5900
54,CWB,2024-06-25,459116,12.80,12.80,12.9000,2024-06-28,13.23,13.23,12.8900
59,CWB,2024-06-25,620872,8.49,8.49,8.4900,2024-06-28,7.65,7.65,7.2900
